## Transforming a `DataFrame`
In this lecture you will learn how to:
- rename columns from a `DataFrame`
- drop columns from a `DataFrame`
- transform a `DataFrame` in a function using `pipe`

In [1]:
import polars as pl

In [2]:
csvFile = "data/titanic.csv"

In [3]:
df = pl.read_csv(csvFile)
df.head(2)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow...","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. ...","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""


## Renaming columns
We can rename columns by passing a `dict` that maps old names to new names.

In [ ]:
(
    df
    .rename({"PassengerId":"ID"})
    .head(2)
)

## Dropping columns

We can drop columns by passing a `list` of column names

In [4]:
(
    df
    .drop(["PassengerId","Pclass"])
    .head(2)
)

Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,str,str,f64,i64,i64,str,f64,str,str
0,"""Braund, Mr. Ow...","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
1,"""Cumings, Mrs. ...","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""


## Re-ordering columns
We can re-order columns with a `list` in `select`.

In this example we re-order the columns in alphabetical order

In [ ]:
(
    df
    .select(sorted(df.columns))
)

## Re-ordering columns in a query

We may want to re-order the columns in a query where we cannot access the `columns` attribute. This may be because in the query:
- we have just read the file
- we have added/removed columns
- we have changed column names

We can still re-order columns using `pipe`.

The `pipe` method allows us to access the `DataFrame` at that point in a query as if we have it assigned to a variable.

In this example we sort the columns alphabetically after reading a CSV

In [ ]:
(
    pl.read_csv(csvFile)
    .pipe(
        lambda tempDf: tempDf.select( sorted(tempDf.columns))
    )
    .head(2)
)

We can also use `pipe` in lazy mode

In [ ]:
(
    pl.scan_csv(csvFile)
    .pipe(
        lambda qdf: qdf.select( sorted(qdf.columns[:3]))
    )
)

The transformations in `pipe` are passed to the query optimiser in lazy mode.

In this example we only use the first three columns in the `select`

In [ ]:
print(
    pl.scan_csv(csvFile)
    .pipe(
        lambda qdf: qdf.select( sorted(qdf.columns[:3]))
    )
    .describe_optimized_plan()
)

The query optimiser sees that only 3 columns are required and sets 
```bash
PROJECT 3/12 COLUMNS
```

## Applying a function to a `DataFrame`
We apply a function to a `DataFrame` using `pipe`.

In this example we define a function `_multiply_floats` where we select all the floating point columns and multiply them by 3

In [ ]:
def _multiply_floats(df: pl.DataFrame, mul: int) -> pl.DataFrame:
    return df.select(pl.col(pl.Float64)) * mul

(
    df
    .pipe(
        _multiply_floats, 
        mul=3)
    .head(3)
)


In [11]:
# can I iteratively run functions, for simulation purposes?
def add_sequential(df: pl.DataFrame, mul: int) -> pl.DataFrame:
    return df.with_column((pl.col("Age") + 1).alias("Age"))

for i in range(2):
    df = (
        df
        .pipe(
            add_sequential,
            mul = 1
        )
        .select(pl.col("Age"))
        .head(3)
    )
    print(df)


shape: (3, 1)
┌──────┐
│ Age  │
│ ---  │
│ f64  │
╞══════╡
│ 23.0 │
│ 39.0 │
│ 27.0 │
└──────┘
shape: (3, 1)
┌──────┐
│ Age  │
│ ---  │
│ f64  │
╞══════╡
│ 24.0 │
│ 40.0 │
│ 28.0 │
└──────┘


## Exercises
In the exercises you will develop your understanding of:
- renaming columns
- dropping columns
- transformations using `pipe`

### Exercise 1
Drop the `Age` and `Fare` columns from the `DataFrame`

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

### Exercise 2
Rename the `Age` column to `age`

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

Rename all column names to lower case. Expand the cell below if you would like a hint

In [ ]:
#Hint: do the renaming inside .pipe
#Hint: use the Python method .lower() on column name strings

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

## Solutions

### Solution to exercise 1
Drop the `Age` and `Fare` columns from the `DataFrame`

In [ ]:
(
    pl.read_csv(csvFile)
    .drop(["Age","Fare"])
    .head(3)
)

### Solution to exercise 2

Rename the `Age` column to `age`

In [ ]:
(
    pl.read_csv(csvFile)
    .rename({"Age":"age"})
    .head(3)
)

Rename all column names to lower case. Expand the cell below if you would like a hint

In [ ]:
#Hint: do the renaming inside .pipe
#Hint: use the Python method .lower() on column name strings

In [ ]:
(
    pl.read_csv(csvFile)
    .pipe(
        lambda df:df.rename({oldCol:oldCol.lower() for oldCol in df.columns})
    )
    .head(3)
)